In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [ ]:
data = pd.read_csv('diabetes.csv')

data.info()
print("row one of dataset:\n",data.head(1))

# Display basic statistics of the dataset
print("Basic Statistics of the Dataset:")
print(data.describe())


In [ ]:
# Separate features (X) and target variable (y)
X = data.drop(columns=['Outcome'])  # Assuming 'Outcome' is the target variable
y = data['Outcome']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y,  random_state=104,  test_size=0.3,  shuffle=True) 

# Print out train and test sets
print('X_train row zero: ') 
print(X_train.head(0)) 
print('') 
print('X_test row zero: ') 
print(X_test.head(0)) 
print('') 
print('y_train ro w zero: ') 
print(y_train.head(0)) 
print('') 
print('y_test row zero: ') 
print(y_test.head(0))

#there is no null value but there is some zerro value that must considet!
#y_train = y_train[X_train.index]  #Adjust y_train accordingly after dropping rows
# Count zero values in the 'SkinThickness' column in the training dataset
zero_skinthickness_count = (X_train['SkinThickness'] == 0).sum()
print("\nNumber of zero values in the 'SkinThickness' column in the training dataset:", zero_skinthickness_count)

# Count zero values in the 'Glucose' column in the training dataset
glucose_zeros_count = (X_train['Glucose'] == 0).sum()
print("Number of zero values in the 'Glucose' column in the training dataset:", glucose_zeros_count)

# Count zero values in the 'BMI' column in the training dataset
zero_bmi_count = (X_train['BMI'] == 0).sum()
print("Number of zero values in the 'BMI' column in the training dataset:", zero_bmi_count)

# the zero value left untouched but it consider in scaling and standardaizing



In [ ]:
# Create a DataFrame with X_train and y_train (The pandas. concat() function concatenates and combines multiple DataFrames or Series into a single, unified DataFrame or Series.)
train_data = pd.concat([X_train, y_train], axis=1)

# Calculate the mean of each feature for outcome 0 and outcome 1
#mean_outcome_0 = train_data[train_data['Outcome'] == 0].mean()
#mean_outcome_1 = train_data[train_data['Outcome'] == 1].mean()

# Fill missing values for 'BMI', 'Glucose', and 'BloodPressure' based on the mean of each feature for the respective outcome groups
#for feature in ['BMI', 'Glucose', 'BloodPressure' , 'SkinThickness','Age']:
    # Fill missing values for outcome 0
 #   X_train.loc[(X_train[feature] == 0) & (y_train == 0), feature] = mean_outcome_0[feature].astype('int64')
    # Fill missing values for outcome 1
 #   X_train.loc[(X_train[feature] == 0) & (y_train == 1), feature] = mean_outcome_1[feature].astype('int64')

# Print the updated X_train with filled missing values
#print("X_train with filled missing values:")
#print(X_train.head(1))


analyze the distribution of features correlations

In [ ]:
# Compute the correlation matrix for X_train
correlation_matrix = X_train.corr()

# Plot the correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Feature Correlation Heatmap')
plt.show()

# Visualize the distribution of each feature in X_train
plt.figure(figsize=(12, 8))
for i, column in enumerate(X_train.columns):
    plt.subplot(3, 3, i+1)
    sns.histplot(X_train[column], kde=True)
    plt.title(column)
plt.tight_layout()
plt.show()

# Visualize pairwise relationships between features in X_train
sns.pairplot(data, hue= 'Outcome' )
plt.suptitle("Pairwise Relationships between Features")
plt.show()

# Visualize boxplots to identify outliers in X_train
plt.figure(figsize=(12, 8))
for i, column in enumerate(X_train.columns):
    plt.subplot(3, 3, i+1)
    sns.boxplot(x=y, y=X_train[column])
    plt.title(column)
plt.tight_layout()
plt.show()

In [ ]:
#Shaping the dataset according to my requirements. The dataset has already been split into training and test sets before the shaping process.

In [ ]:
import pandas as pd

# Assuming X_train is your dataset containing features
# Let's say X_train has columns ['BMI', 'Glucose', 'BloodPressure', 'SkinThickness', 'Age']

# Create a new DataFrame to store the extended dataset
X_train_ex = X_train.copy()
X_test_ex  =X_test.copy()
data_ex = data.copy()
# List of pairs of features to combine
feature_pairs = [('BMI', 'Glucose'), ('BMI', 'BloodPressure'), ('BMI', 'SkinThickness'), ('BMI', 'Age'),
                 ('Glucose', 'BloodPressure'), ('Glucose', 'SkinThickness'), ('Glucose', 'Age'),
                 ('BloodPressure', 'SkinThickness'), ('BloodPressure', 'Age'),
                 ('SkinThickness', 'Age')]

# Iterate through each pair of features
for feat1, feat2 in feature_pairs:
    # Create new feature by combining the values of two existing features
    new_feature_name = feat1 + 'Plus' + feat2
    X_train_ex[new_feature_name] = X_train_ex[feat1] + X_train_ex[feat2]
    X_test_ex[new_feature_name] = X_test_ex[feat1] + X_test_ex[feat2]
    data_ex[new_feature_name]= data_ex[feat1] + data_ex[feat2]

# Now X_train_ex contains the original features plus the new features created by combining pairs of existing features
print(data_ex.head(1))

# Visualize pairwise relationships between features in X_train
sns.pairplot(data_ex, hue= 'Outcome' )
plt.suptitle("Pairwise Relationships between Features")
plt.show()


In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

 #Selecting specific features
selected_features = ['BMI', 'Age', 'Glucose', 'BloodPressure','DiabetesPedigreeFunction']
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Train Support Vector Machine (SVM) model
svm_model = SVC(C= 10000, kernel= 'poly',gamma='scale')
svm_model.fit(X_train_selected, y_train)

# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=1000, max_depth=100,criterion= 'entropy',
        min_samples_split=6, random_state=42, bootstrap= True,
        max_features=4
        )
rf_model.fit(X_train_selected, y_train)

# Train Logistic Regression model with increased max_iter
lr_model = LogisticRegression(max_iter=10000, solver='lbfgs')
lr_model.fit(X_train_selected, y_train)

# Make predictions on the test set
svm_test_pred = svm_model.predict(X_test_selected)
rf_test_pred = rf_model.predict(X_test_selected)
lr_test_pred = lr_model.predict(X_test_selected)

# Evaluate the models on the test set
svm_test_accuracy = accuracy_score(y_test, svm_test_pred)
rf_test_accuracy = accuracy_score(y_test, rf_test_pred)
lr_test_accuracy = accuracy_score(y_test, lr_test_pred)

print("Test Set Accuracy:")
print("Support Vector Machine (SVM) Accuracy:", svm_test_accuracy)
print("Random Forest Accuracy:", rf_test_accuracy)
print("Logistic Regression Accuracy:", lr_test_accuracy)


In [ ]:
from sklearn.metrics import classification_report

# Evaluate the models on the test set
print("Test Set Evaluation:")
print("Support Vector Machine (SVM):")
print(classification_report(y_test, svm_test_pred))

print("Random Forest:")
print(classification_report(y_test, rf_test_pred))

print("Logistic Regression:")
print(classification_report(y_test, lr_test_pred))




In [ ]:
# Train Support Vector Machine (SVM) model
selected_features_ex = ['BMIPlusGlucose','GlucosePlusSkinThickness','GlucosePlusAge','BMIPlusBloodPressure','BMIPlusAge']
X_train_ex_selected = X_train_ex[selected_features_ex]
X_test_ex_selected = X_test_ex[selected_features_ex]

# Train Support Vector Machine (SVM) model
svm_model = SVC(C= 10, kernel= 'rbf',gamma='scale')
svm_model.fit(X_train_ex_selected, y_train)


# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=1000, max_depth=100, criterion='entropy',
                                  min_samples_split=2, random_state=42, bootstrap=True,
                                  max_features=32)
rf_model.fit(X_train_ex_selected, y_train)

# Train Logistic Regression model with increased max_iter
lr_model = LogisticRegression(max_iter=10000, solver='lbfgs')
lr_model.fit(X_train_ex_selected, y_train)
# Test Support Vector Machine (SVM) model
svm_test_pred = svm_model.predict(X_test_ex_selected )

# Test Random Forest model
rf_test_pred = rf_model.predict(X_test_ex_selected )

# Test Logistic Regression model
lr_test_pred = lr_model.predict(X_test_ex_selected )





In [ ]:
from sklearn.metrics import accuracy_score

# Compute accuracy score for each model
svm_test_accuracy = accuracy_score(y_test, svm_test_pred)
rf_test_accuracy = accuracy_score(y_test, rf_test_pred)
lr_test_accuracy = accuracy_score(y_test, lr_test_pred)

print("Test Set Accuracy:")
print("Support Vector Machine (SVM) Accuracy:", svm_test_accuracy)
print("Random Forest Accuracy:", rf_test_accuracy)
print("Logistic Regression Accuracy:", lr_test_accuracy)

# Evaluate the models on the test set
print("Test Set Evaluation:")
print("Support Vector Machine (SVM):")
print(classification_report(y_test, svm_test_pred))

print("Random Forest:")
print(classification_report(y_test, rf_test_pred))

print("Logistic Regression:")
print(classification_report(y_test, lr_test_pred))

